### Učitavanje i predprocesiranje podataka
Započinjemo učitavanjem skupa podataka i izvođenje početnih koraka predprocesiranja kao što je filtriranje ćelija i gena, i izračunavanje QC metrike 

In [ ]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
from sklearn.metrics import davies_bouldin_score


adata = ad.read_h5ad('GTEX-1HSMQ-5005.h5ad')


sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)


### Vizualizacija 

In [ ]:
sc.pl.violin(adata, ["n_genes_by_counts", "total_counts", "pct_counts_mt"], jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts")

### Filtriranje i normalizacija podataka 
Filtriramo ćelije na osnovu QC metrike i normalizujemo podatke kako bismo osigurali upotrebljivost među uzorcima.

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 2000, :]

# Normalizacija podataka
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.layers['log_norm'] = adata.X.copy()

### Identifikacija visoko varijabilnih gena i transformacija podataka
Identifikujemo visoko varijabilne gene i izvodimo transformaciju podataka kako bismo uklonili neželjenu varijabilnost.

In [ ]:

sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)


### Redukcija dimenzionalnosti i klasterovanje

In [ ]:

sc.tl.pca(adata, svd_solver='arpack')


sc.tl.umap(adata)


sc.tl.leiden(adata)


### Računanje Davies-Bouldin score-a i poređenje

In [ ]:

umap_clusters = adata.obs['leiden'].values
umap_embedding = adata.obsm['X_umap']
db_score_normalized = davies_bouldin_score(umap_embedding, umap_clusters)

print(f"Davies-Bouldin skor sa normalizacijom: {db_score_normalized}")


adata_raw = adata.raw.to_adata()  


sc.tl.pca(adata_raw, svd_solver='arpack')
sc.tl.umap(adata_raw)
sc.tl.leiden(adata_raw)


umap_clusters_raw = adata_raw.obs['leiden'].values
umap_embedding_raw = adata_raw.obsm['X_umap']
db_score_raw = davies_bouldin_score(umap_embedding_raw, umap_clusters_raw)

print(f"Davies-Bouldin skor bez normalizacije: {db_score_raw}")


if db_score_normalized < db_score_raw:
    print("Klasterovanje sa normalizacijom je rezultovalo bolje definisanim klasterima.")
else:
    print("Klasterovanje bez normalizacije je rezultovalo bolje definisanim klasterima.")
